Будем использовать физическими величинами с измерениями. Модуль PhysicalQuantities сломался, не обновлялся два года, поэтому придется попробовать новое.

In [8]:
from pint import UnitRegistry
from uncertainties import ufloat

In [9]:
ureg = UnitRegistry()
length = ufloat(10, 0.5) * ureg.meter
print(length)  # (10 ± 0.5) m


10.0+/-0.5 meter
